In [27]:
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.static import teams
import pandas as pd
import time

year=1996
# Fetch advanced stats for 2000-01 season
team_advanced_stats = leaguedashteamstats.LeagueDashTeamStats(
    season=f"{year}-{str(year + 1)[-2:]}",
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Advanced'
).get_data_frames()[0]

nba_champ = teams._find_teams_by_championship_year(year)

team_ratings_df = pd.DataFrame({
    "Team": list(team_advanced_stats['TEAM_NAME']),
    "Off_Rating": list(team_advanced_stats['OFF_RATING']),       
    "Def_Rating": list(team_advanced_stats['DEF_RATING']),
    "Year": [year]*len(team_advanced_stats),   
    "NBA Champion": [nba_champ==team for team in team_advanced_stats['TEAM_NAME']]
})


In [28]:
for year in range(1997,2026):
    
    team_advanced_stats = leaguedashteamstats.LeagueDashTeamStats(
    season=f"{year-1}-{str(year)[-2:]}",
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Advanced'
    ).get_data_frames()[0]
    
    if year<2024:
        nba_champ = teams._find_teams_by_championship_year(year)
    elif year==2024:
        nba_champ='Boston Celtics'
    else:
        nba_champ=""
    team_ratings_df_2 = pd.DataFrame({
        "Team": list(team_advanced_stats['TEAM_NAME']),
        "Off_Rating": list(team_advanced_stats['OFF_RATING']),       
        "Def_Rating": list(team_advanced_stats['DEF_RATING']),
        "Year": [year]*len(team_advanced_stats),  
        "NBA Champion": [nba_champ==team for team in team_advanced_stats['TEAM_NAME']]
    })

    team_ratings_df=pd.concat([team_ratings_df,team_ratings_df_2],ignore_index=True)
    time.sleep(0.1)
    
    
team_ratings_df

,Team,Off_Rating,Def_Rating,Year,NBA Champion
0,Atlanta Hawks,106.4,100.3,1996,False
1,Boston Celtics,102.9,110.1,1996,False
2,Charlotte Hornets,109.1,107.1,1996,False
3,Chicago Bulls,112.4,100.7,1996,True
4,Cleveland Cavaliers,102.7,100.8,1996,False
...,...,...,...,...,...
886,Sacramento Kings,114.8,113.8,2025,False
887,San Antonio Spurs,111.3,111.9,2025,False
888,Toronto Raptors,112.0,115.0,2025,False
889,Utah Jazz,108.8,119.0,2025,False


Normalize the data to get a 0 to 1 scale for both OR and DR

In [29]:
for year in range(1996,2026):
    df = team_ratings_df[team_ratings_df['Year']==year].copy()
    
    Max_Off_Rat = df['Off_Rating'].max()
    Max_Def_Rat = df['Def_Rating'].max()

    Min_Off_Rat = df['Off_Rating'].min()
    Min_Def_Rat = df['Def_Rating'].min()

    
    for i in range(len(df)):
        Off_Rat = df.iloc[i]['Off_Rating']
        Def_Rat = df.iloc[i]['Def_Rating']
        
        normalized_off_rat = (Off_Rat-Min_Off_Rat) / (Max_Off_Rat-Min_Off_Rat) if Max_Off_Rat!=Min_Off_Rat else 0
        normalized_def_rat = (Def_Rat-Min_Def_Rat) / (Max_Def_Rat-Min_Def_Rat) if Max_Def_Rat!=Min_Def_Rat else 0
        
        team_ratings_df.loc[df.index[i], 'Normalized_Off_Rating'] = normalized_off_rat
        team_ratings_df.loc[df.index[i], 'Normalized_Def_Rating'] = 1-normalized_def_rat
               

In [32]:
team_ratings_df[team_ratings_df['Year']==2025]

,Team,Off_Rating,Def_Rating,Year,NBA Champion,Normalized_Off_Rating,Normalized_Def_Rating
861,Atlanta Hawks,111.5,113.4,2025,False,0.450867,0.370861
862,Boston Celtics,120.7,110.5,2025,False,0.982659,0.562914
863,Brooklyn Nets,112.7,116.1,2025,False,0.520231,0.192053
864,Charlotte Hornets,108.9,113.7,2025,False,0.300578,0.350993
865,Chicago Bulls,113.4,117.6,2025,False,0.560694,0.092715
866,Cleveland Cavaliers,120.5,110.4,2025,False,0.971098,0.569536
867,Dallas Mavericks,116.5,110.4,2025,False,0.739884,0.569536
868,Denver Nuggets,115.6,113.9,2025,False,0.687861,0.337748
869,Detroit Pistons,110.6,113.5,2025,False,0.398844,0.364238
870,Golden State Warriors,113.7,108.0,2025,False,0.578035,0.728477


In [33]:
team_ratings_df.to_csv("1996-2025 team ratings.csv", index=True)